# COVID-19 Drug Repurposing Example
This example shows how to do drug repurposing using DRKG even with the pretrained model.

## Collecting COVID-19 related disease
At the very beginning we need to collect a list of disease of Corona-Virus(COV) in DRKG. We can easily use the Disease ID that DRKG uses for encoding the disease. Here we take all of the COV disease as target.

In [117]:
COV_disease_list = [
'Disease::SARS-CoV2 E',
'Disease::SARS-CoV2 M',
'Disease::SARS-CoV2 N',
'Disease::SARS-CoV2 Spike',
'Disease::SARS-CoV2 nsp1',
'Disease::SARS-CoV2 nsp10',
'Disease::SARS-CoV2 nsp11',
'Disease::SARS-CoV2 nsp12',
'Disease::SARS-CoV2 nsp13',
'Disease::SARS-CoV2 nsp14',
'Disease::SARS-CoV2 nsp15',
'Disease::SARS-CoV2 nsp2',
'Disease::SARS-CoV2 nsp4',
'Disease::SARS-CoV2 nsp5',
'Disease::SARS-CoV2 nsp5_C145A',
'Disease::SARS-CoV2 nsp6',
'Disease::SARS-CoV2 nsp7',
'Disease::SARS-CoV2 nsp8',
'Disease::SARS-CoV2 nsp9',
'Disease::SARS-CoV2 orf10',
'Disease::SARS-CoV2 orf3a',
'Disease::SARS-CoV2 orf3b',
'Disease::SARS-CoV2 orf6',
'Disease::SARS-CoV2 orf7a',
'Disease::SARS-CoV2 orf8',
'Disease::SARS-CoV2 orf9b',
'Disease::SARS-CoV2 orf9c',
'Disease::MESH:D045169',
'Disease::MESH:D045473',
'Disease::MESH:D001351',
'Disease::MESH:D065207',
'Disease::MESH:D028941',
'Disease::MESH:D058957',
'Disease::MESH:D006517'
]

## Candidate drugs
Now we use FDA-approved drugs in Drugbank as candidate drugs. (we exclude drugs with molecule weight < 250) The drug list is in infer\_drug.tsv

In [76]:
import csv

# Load entity file
drug_list = []
with open("./infer_drug.tsv", newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile, delimiter='\t', fieldnames=['drug','ids'])
    for row_val in reader:
        drug_list.append(row_val['drug'])

## Treatment relation

In [77]:
treatment = 'GNBR::T::Compound:Disease'

## Get pretrained model
We can directly use the pretrianed model to do drug repurposing.

In [78]:
import pandas as pd
import numpy as np
import sys
sys.path.insert(1, '../utils')
from utils import download_and_extract
download_and_extract()

In [79]:
entity_idmap_file = '../data/drkg/embed/entities.tsv'
relation_idmap_file = '../data/drkg/embed/relations.tsv'

## Get embeddings for diseases and drugs

In [98]:
# Get drugname/disease name to entity ID mappings
entity_map = {}
entity_id_map = {}
relation_map = {}
with open(entity_idmap_file, newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile, delimiter='\t', fieldnames=['name','id'])
    for row_val in reader:
        entity_map[row_val['name']] = int(row_val['id'])
        entity_id_map[int(row_val['id'])] = row_val['name']
        
with open(relation_idmap_file, newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile, delimiter='\t', fieldnames=['name','id'])
    for row_val in reader:
        relation_map[row_val['name']] = int(row_val['id'])
        
# handle the ID mapping
drug_ids = []
disease_ids = []
for drug in drug_list:
    drug_ids.append(entity_map[drug])
    
for disease in COV_disease_list:
    disease_ids.append(entity_map[disease])
    
treatment_rid = relation_map[treatment]

In [99]:
# Load embeddings
import torch as th
entity_emb = np.load('../data/drkg/embed/DRKG_TransE_l2_entity.npy')
rel_emb = np.load('../data/drkg/embed/DRKG_TransE_l2_relation.npy')

drug_ids = th.tensor(drug_ids).long()
disease_ids = th.tensor(disease_ids).long()
treatment_rid = th.tensor(treatment_rid)

drug_emb = th.tensor(entity_emb[drug_ids])
treatment_emb = th.tensor(rel_emb[treatment_rid])

## Drug Repurposing Based on Edge Score
We use following algorithm to calculate the edge score. Note, here we use logsigmiod to make all scores < 0. The larger the score is, the stronger the $h$ will have $r$ with $t$.

$\mathbf{d} = \gamma - ||\mathbf{h}+\mathbf{r}-\mathbf{t}||_{2}$

$\mathbf{score} = \log\left(\frac{1}{1+\exp(\mathbf{-d})}\right)$

When doing drug repurposing, we only use GNBR::T::Compound:Disease relation.

In [119]:
import torch.nn.functional as fn

gamma=12.0
def transE_l2(head, rel, tail):
    score = head + rel - tail
    return gamma - th.norm(score, p=2, dim=-1)

scores_per_disease = []
dids = []
for disease_id in disease_ids:
    disease_emb = entity_emb[disease_id]
    score = fn.logsigmoid(transE_l2(drug_emb, treatment_emb, disease_emb))
    scores_per_disease.append(score)
    dids.append(drug_ids)
    
scores = th.cat(scores_per_disease)
dids = th.cat(dids)

In [120]:
# sort scores in decending order
idx = th.flip(th.argsort(scores), dims=[0])
scores = scores[idx].numpy()
dids = dids[idx].numpy()

### Now we output proposed treatments

In [121]:
_, unique_indices = np.unique(dids, return_index=True)
top100_indices = np.sort(unique_indices)[:100]
proposed_dids = dids[top100_indices]
proposed_scores = scores[top100_indices]

Now we list the pairs of in form of (drug, treat, disease, score) 

We select top 100 relevent drugs according the edge score

In [122]:
for i in range(100):
    drug = int(proposed_dids[i])
    score = proposed_scores[i]
    
    print("{}\t{}".format(entity_id_map[drug], score))

Compound::DB00811	-0.21358221769332886
Compound::DB00982	-0.7457163333892822
Compound::DB01082	-0.80478835105896
Compound::DB00563	-0.8762229681015015
Compound::DB00635	-0.8786484599113464
Compound::DB00681	-0.9245818853378296
Compound::DB00290	-0.9402258396148682
Compound::DB00860	-0.9569648504257202
Compound::DB00249	-0.9634518027305603
Compound::DB01222	-0.9871338605880737
Compound::DB00993	-0.9912917613983154
Compound::DB00512	-1.0083342790603638
Compound::DB00624	-1.0209006071090698
Compound::DB01024	-1.0232834815979004
Compound::DB01234	-1.025983214378357
Compound::DB01004	-1.0309269428253174
Compound::DB00688	-1.03281569480896
Compound::DB00091	-1.0585496425628662
Compound::DB00558	-1.0633755922317505
Compound::DB00741	-1.0851504802703857
Compound::DB00783	-1.0877355337142944
Compound::DB00552	-1.0897188186645508
Compound::DB01394	-1.0996224880218506
Compound::DB00131	-1.1011534929275513
Compound::DB01212	-1.1101065874099731
Compound::DB00415	-1.1111209392547607
Compound::DB0011

### Check Clinial Trial Drugs
There are seven clinial trial drugs hit in top100. (Note: Ribavirin exists in DRKG as a treatment for SARS)

In [123]:
clinical_drugs_file = './COVID19_clinical_trial_drugs.tsv'
clinical_drug_map = {}
with open(clinical_drugs_file, newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile, delimiter='\t', fieldnames=['id', 'drug_name','drug_id'])
    for row_val in reader:
        clinical_drug_map[row_val['drug_id']] = row_val['drug_name']
        
for i in range(100):
    drug = entity_id_map[int(proposed_dids[i])][10:17]
    if clinical_drug_map.get(drug, None) is not None:
        score = proposed_scores[i]
        print("[{}]{}".format(i, clinical_drug_map[drug]))

[0]Ribavirin
[14]Dexamethasone
[22]Colchicine
[35]Methylprednisolone
[49]Oseltamivir
[71]Chloroquine
[91]Thalidomide
